In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
import scipy
import statsmodels.stats as stats
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
path = "../../data/"

In [3]:
df = pd.read_csv(path + "housing2019.csv", encoding='latin-1')
df_crime = pd.read_csv(path + "King_County_Sheriff_s_Office_-_Incident_Dataset.csv")
df_pop_est = pd.read_csv(path + "data.csv")

/Users/calvintirrell/opt/anaconda3/envs/linreg-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [48]:
# df.drop(labels='Unnamed: 0', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29621 entries, 0 to 29620
Columns: 156 entries, Major_x to OtherProblems
dtypes: bool(1), float64(5), int64(90), object(60)
memory usage: 35.1+ MB


In [5]:
# df_crime = df_crime[['city','zip']]
df_crime.head()

,case_number,incident_datetime,incident_type,FCR,address_1,city,state,zip,created_at,updated_at,hour_of_day,day_of_week,Incident Block Location,KCSO Patrol Districts,KCSO Reporting Districts
0,C19045456,11/27/2019 02:10:00 PM,Other,377,8700 Block 250TH AVE NE,REDMOND,WA,98053,12/02/2019 11:55:00 PM,12/03/2019 01:47:00 AM,14,Wednesday,POINT (-122.00566700000002 47.679241),15.0,254.0
1,C19045392,11/27/2019 08:00:00 AM,Traffic,427,BECKLER RD & MP 4.6,SKYKOMISH,WA,98288,12/02/2019 10:25:00 PM,12/03/2019 12:16:00 AM,8,Wednesday,NaN,NaN,NaN
2,C19047596,11/20/2019 11:38:00 AM,Property Crime,331,NFD 6028 & HWY 2,BARING,WA,5474,12/17/2019 11:15:00 PM,12/17/2019 11:56:00 PM,11,Wednesday,NaN,NaN,NaN
3,C19044352,11/19/2019 02:38:00 PM,Drugs,200,1 Block AVE S & S KING ST,SEATTLE,WA,98144,12/02/2019 05:05:00 PM,12/02/2019 06:56:00 PM,14,Tuesday,NaN,NaN,NaN
4,C19044243,11/18/2019 11:06:00 PM,Traffic,404,I,KENT,WA,NaN,11/26/2019 08:55:00 PM,11/26/2019 10:46:00 PM,23,Monday,NaN,NaN,NaN


In [10]:
df_crime_count = df_crime.copy()

In [13]:
df_crime_count = pd.DataFrame(df_crime.groupby(by='city').size(), columns=['crime_count'])
df_crime_count

,crime_count
city,
ABERDEEN,1
ALGONA,8
ASTORIA,1
AUBURN,1862
BARING,22
...,...
WATERBURY,1
WHITE CENTER,254
WOODINVILLE,482


In [8]:
df_pop_est['Name'] = df_pop_est['Name'].str.upper()
# df_pop_est = df_pop_est[['Name', 'PopTotal']]
df_pop_est.head()

,Name,Census,PopTotal,Growth,GrowthRate
0,SEATTLE,608659,704352,95693,0.1572
1,BELLEVUE,127887,141400,13513,0.1057
2,KENT,118593,127514,8921,0.0752
3,RENTON,91900,100953,9053,0.0985
4,FEDERAL WAY,89306,96757,7451,0.0834


In [24]:
df_pop_crime = df_crime_count.merge(df_pop_est, how='left', left_on='city', right_on='Name')
df_pop_crime.dropna(inplace=True)
df_pop_crime['crime_rate'] = df_pop_crime['crime_count'] / df_pop_crime['PopTotal']
df_pop_crime.head()

,crime_count,Name,Census,PopTotal,Growth,GrowthRate,crime_rate
1,8,ALGONA,3014.0,3156.0,142.0,0.0471,0.002535
6,1,BEAUX ARTS VILLAGE,299.0,326.0,27.0,0.0903,0.003067
7,74,BELLEVUE,127887.0,141400.0,13513.0,0.1057,0.000523
9,47,BLACK DIAMOND,4151.0,4384.0,233.0,0.0561,0.010721
18,267,BURIEN,48072.0,50997.0,2925.0,0.0608,0.005236


In [49]:
df_crime_pop = df_crime.merge(df_pop_crime, how='left', left_on='city', right_on='Name')
df_crime_pop = df_crime_pop[['zip', 'crime_rate']]
df_crime_pop.dropna(inplace=True)

In [50]:
df_crime_pop

,zip,crime_rate
0,98053,0.010679
1,98288,0.194175
3,98144,0.011021
5,98104,0.011021
6,98053,0.010679
...,...,...
20441,98101,0.011021
20442,98104,0.011021
20443,98146,0.011021
20446,98030,0.009458


In [51]:
df.merge(df_crime_pop, how='left', left_on='ZipCode', right_on='zip')

,Major_x,Minor_x,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,...,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems,zip,crime_rate
0,8700,215,1,1,17029 106TH AVE SE 98055,17029,,,106TH,AVE,...,N,N,N,N,N,N,N,N,98055,0.02098
1,8700,215,1,1,17029 106TH AVE SE 98055,17029,,,106TH,AVE,...,N,N,N,N,N,N,N,N,98055,0.02098
2,8700,215,1,1,17029 106TH AVE SE 98055,17029,,,106TH,AVE,...,N,N,N,N,N,N,N,N,98055,0.02098
3,8700,215,1,1,17029 106TH AVE SE 98055,17029,,,106TH,AVE,...,N,N,N,N,N,N,N,N,98055,0.02098
4,8700,215,1,1,17029 106TH AVE SE 98055,17029,,,106TH,AVE,...,N,N,N,N,N,N,N,N,98055,0.02098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4217597,403950,490,1,1,15939 SE 1ST ST 98008,15939,,SE,1ST,ST,...,N,N,N,N,N,N,N,N,NaN,NaN
4217598,403970,280,1,1,206 174TH PL NE 98008,206,,,174TH,PL,...,N,N,N,N,N,N,N,N,NaN,NaN
4217599,404080,180,1,1,1249 169TH AVE NE 98008,1249,,,169TH,AVE,...,N,N,N,N,N,N,N,N,NaN,NaN
4217600,404660,310,1,1,31727 NE 114TH ST 98014,31727,,NE,114TH,ST,...,N,N,N,N,N,N,N,N,NaN,NaN


In [14]:
# fsm_3 = ols(formula="SalePrice ~ SqFtTotLiving + TidelandShoreland + SqFtOpenPorch + SqFtDeck + Area + Bedrooms + Bath3qtrCount + BathFullCount + LakeSammamish + LakeWashington + SeattleSkyline + WaterSystem + SewerSystem + SqFtGarageBasement + SqFtGarageAttached + YrBuilt + YrRenovated", data=df).fit()

fsm_3 = ols(formula = "SalePrice ~ TidelandShoreland + SqFtOpenPorch + SqFtDeck + Area + Bath3qtrCount + LakeSammamish + LakeWashington + SeattleSkyline + SqFtGarageBasement + SqFtGarageAttached + YrRenovated", data=df).fit()


In [15]:
fsm_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.252
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     906.6
Date:                Thu, 05 Mar 2020   Prob (F-statistic):               0.00
Time:                        08:57:23   Log-Likelihood:            -4.2751e+05
No. Observations:               29621   AIC:                         8.550e+05
Df Residuals:                   29609   BIC:                         8.551e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           5.748e+05   6252.409     91.931      0.000    5.63e+05    5.87e+05
TidelandShoreland   1.912e+05   1.74e+04     11.003      0.000    1.57e+05    2.25e+05
SqFtOpenPorch        814.2321     18.625     43.718      0.000     777.727     850.737
SqFtDeck             289.5215     12.849     22.533      0.000     264.337     314.706
Area               -1473.6424    100.110    -14.720      0.000   -1669.863   -1277.422
Bath3qtrCount       1.003e+05   4051.295     24.748      0.000    9.23e+04    1.08e+05
LakeSammamish       3.306e+05   1.36e+04     24.245      0.000    3.04e+05    3.57e+05
LakeWashington       2.56e+05   7652.253     33.459      0.000    2.41e+05    2.71e+05
SeattleSkyline      2.281e+05    1.1e+04     20.657      0.000    2.06e+05     2.5e+05
SqFtGarageBasement   128.6437     14.822      8.679      0.000      99.593     157.695
SqFtGarageAttached   300.9520     10.195     29.520      0.000     280.970     320.934
YrRenovated           94.6325      6.371     14.853      0.000      82.144     107.121
==============================================================================
Omnibus:                    14236.368   Durbin-Watson:                   1.671
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           263279.947
Skew:                           1.868   Prob(JB):                         0.00
Kurtosis:                      17.120   Cond. No.                     3.01e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.01e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
rainbow_statistic, rainbow_p_value = linear_rainbow(fsm_3)
print("Rainbow statistic:", rainbow_statistic)
print("Rainbow p-value:", rainbow_p_value)

Rainbow statistic: 1.082436996229488
Rainbow p-value: 7.233907068135861e-07
